#### **Data Science na Prática**
*by [Wilkne Maia](https://wilkne)*

---

<h1>Parquet: O Formato que Está Transformando a Análise de Dados</h1>

No universo da análise de dados, a eficiência e a velocidade são cruciais. É aqui que o formato Parquet entra em cena, revolucionando a forma como armazenamos e processamos informações. Mas o que torna o Parquet tão especial?

O Parquet é um formato de armazenamento de dados colunar, open-source, que se destaca por otimizar a leitura e o processamento de grandes volumes de dados. Em vez de armazenar os dados linha a linha, como em formatos tradicionais (CSV, por exemplo), o Parquet organiza as informações por colunas. Essa diferença, aparentemente sutil, gera um impacto enorme na performance de consultas e análises.

**Por que o Parquet é tão relevante?**
* **Eficiência de Leitura:** Ao armazenar dados por coluna, o Parquet permite que o sistema leia apenas as colunas necessárias para a análise, reduzindo drasticamente o tempo de I/O (entrada/saída) e o uso de recursos.
* **Compressão Otimizada:** O formato é altamente compressível, o que significa arquivos menores e mais rápidos de transferir e armazenar, além de menor consumo de espaço em disco.
* **Compatibilidade:** O Parquet é amplamente suportado por diversas ferramentas e plataformas de processamento de dados, como Spark, Hadoop, AWS Athena, Google BigQuery e muitas outras.
* **Suporte a Schema:** Ele armazena o esquema dos dados, facilitando a leitura e interpretação das informações.

**Em resumo:** O Parquet é um aliado poderoso para profissionais que lidam com grandes volumes de dados, seja em análise exploratória, data science, machine learning ou outras áreas. Sua capacidade de otimizar o processamento e o armazenamento o torna uma peça-chave no ecossistema de dados moderno.

## Importar os Dados

In [ ]:
# instalar Parquet e Pandas Profiling
%!pip install parquet
%!pip install pandas

In [1]:
# importar os pacotes necessários
import pandas as pd
import os

In [2]:
# importar os dados do arquivo csv para um DataFrame no Pandas
df = pd.read_csv('dados/PFW_2021_public.csv')

## Importar os Dados

In [ ]:
# importar os dados do DataFrame e salva em um arquivo Parquet
df.to_parquet('dados/PFW_2021_public.parquet')

In [ ]:
df_parquet = pd.read_parquet('dados/PFW_2021_public.parquet')
df_parquet.shape
df_parquet.head(5)

In [5]:
# https://stackoverflow.com/questions/2104080/how-do-i-check-file-size-in-python
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0

In [ ]:
csv_em_bytes = os.path.getsize('PFW_2021_public.csv')
convert_bytes(csv_em_bytes)

In [ ]:
%%time

df = pd.read_csv('PFW_2021_public.csv')
df.shape

df.head()

In [11]:
df.to_parquet('PFW_2021_public.parquet')

In [ ]:
parquet_em_bytes = os.path.getsize('PFW_2021_public.parquet')
convert_bytes(parquet_em_bytes)

In [ ]:
%%time

df_pfw_2021_parquet = pd.read_parquet('PFW_2021_public.parquet')
df_pfw_2021_parquet.shape

In [14]:
del df_pfw_2021_parquet

In [ ]:
df_pfw_2021.to_parquet('PFW_2021_public.parquet.gzip', compression='gzip')

parquet_gzip_em_bytes = os.path.getsize('PFW_2021_public.parquet.gzip')
convert_bytes(parquet_gzip_em_bytes)